In [1]:
import sympy as sp

from symqups import s
from symqups.objects.scalars import q, p, alpha, alphaD, W
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho
from symqups.utils.algebra import define
from symqups.operations.quantization import s_quantize

a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
x = sp.Symbol("x")

In [ ]:
inpt = 5*x*sp.exp(x)*ad**2*a**3

s